In [63]:
# from mshr

from __future__ import print_function
from fenics import BoxMesh, Point, DirichletBC, FacetNormal, RectangleMesh
from fenics import FunctionSpace, VectorFunctionSpace
from fenics import TrialFunction, TestFunction
from fenics import Expression, Function, Constant
from fenics import dot, inner, Identity, sym
from fenics import grad, nabla_grad, tr
from ufl import nabla_div
from fenics import sqrt
from fenics import project
from fenics import dx, ds
from fenics import dX, dS
from fenics import solve, interpolate
from fenics import plot
from fenics import File
from fenics import lhs, rhs, assemble
from fenics import Mesh, UnitSquareMesh
from dolfin import near, SubDomain
from fenics import MeshFunction
from fenics import Measure
from ufl import Coefficient, Coefficients
# from dolfin import MeshFunction
from dolfin import info
# import dolfin


In [64]:
import matplotlib.pyplot as plt
%matplotlib inline

In [65]:
line = './results/v0/'


In [66]:
save = 1


In [67]:
mu1 = Constant(0.8e11)
mu2 = Constant(0.5e11)
lambda1 = Constant(1.25e11)
lambda2 = Constant(0.65e11)
k1 = Constant(200.)
k2 = Constant(100.)
beta1 = Constant(6.0e-6)
beta2 = Constant(3.0e-6)
alphaAir = Constant(50.)
TAir = Constant(300.)
THot = Constant(450.)
T0 = Constant(320.)

# TAir = Constant(20.)
# THot = Constant(150.)
# T0 = Constant(40.)

# Finite element order
eo = 1

polynom = 'CG'

In [68]:
THot.values()

array([450.])

In [69]:
# Material settings
rho = 7800  # [kg/m3] density
# k = 50  # [W/(m*K)] thermal conductivity
Cp = 525  # [J/(kg*K)] heat capacity
E = 2.0e011  # [Pa] Young modulus
nu = 0.3  # Poisson coefficient
lambda_ = (nu * E) / ((1 + nu) * (1 - 2 * nu))  # Lamé's first parameter
mu = E / (2 * (1 + nu))  # Lamé's second parameter
# alpha = 2.0e-05  # [1/K] thermal expansion coefficient



#coefficient of slope
c1 = 100_000
# coordinate "y", where is characteristics change
c2 = 0.002
# "stairs" function
# phi_ = '1/(1+exp(-c1 * (x[1] - c2)))'
# phi =  Expression(phi_,  c1 = c1, c2 = c2, degree=eo)

# 1 - bottom thin layer
# 2 - top layer

lambda1 = 1.25e11
lambda2 = 0.65e11
# lambda_ = lambda2 * phi + lambda1 * (1-phi)
lambda_ = Expression("x[2] > 0.002 ? lambda1 : lambda2", lambda1= lambda1, lambda2 = lambda2, degree=eo)
# cc = Expression("x[0] > 0.5 and x[1] > 0.5 ? c : 1", c = c, degree=p+2)


mu1 = 0.8e11
mu2 =0.5e11
# mu = mu2 * phi + mu1 * (1-phi)
mu = Expression("x[2] > 0.002 ? mu1 : mu2", mu1= mu1, mu2 = mu2, degree=eo)


beta1 = 6.0e-06
beta2 = 3.0e-06
# alpha = beta2 * phi + beta1 * (1-phi)
alpha = Expression("x[2] > 0.002 ? beta1 : beta2", beta1= beta1, beta2 = beta2, degree=eo)


k1 = 200  # [W/(m*K)] thermal conductivity on bottom layer
k2 = 100  # [W/(m*K)] thermal conductivity on top layer

# k = k2 * phi + k1 * (1-phi)
k = Expression("x[2] > 0.002 ? k1 : k2", k1= k1, k2 = k2, degree=eo)


# Domain settings
L_x = 0.02  # [m] domain width (x, z)
L_z = 0.012  # [m] domain height
L_y = 0.01  # [m] domain depth
resW = 50  # mesh resolution (x, z)-directions (number of points)
resH = 50  # mesh y-resolution
resD = resW

print('Domain size:', L_x, '[m] x', L_y, '[m] y', L_z, '[m] z')

# Finite element order
# eo = 1

Domain size: 0.02 [m] x 0.01 [m] y 0.012 [m] z


In [70]:
text = str(fr"polynom = {polynom}  eo = {eo} THOt = {THot.values()}")
if save:
    print(text.encode('utf-8'),file=open(f"{line}par.txt",'w'))

In [71]:
# mesh = Mesh('./mesh/v0/CURRENT/composite_simplified.xml')
mesh = Mesh('./composite_fenics/composite.xml')
fd = MeshFunction('size_t', mesh, './composite_fenics/composite_facet_region.xml')
pd = MeshFunction('size_t', mesh, './composite_fenics/composite_physical_region.xml')

# mesh = Mesh('./mesh/classic_meshing/v1/composite_fenics/composite.xml')
# mesh = Mesh('./mesh/composite.xml')
# mesh = UnitSquareMesh(32,32,L_x,L_y, 'left/right')
# mesh = RectangleMesh(Point(0.0, 0.0), Point(L_x, L_y), resW, resH,'left/right')
# mesh = RectangleMesh(Point(0.0, 0.0), Point(L_x, L_y), resW, resH,'left/right')


In [72]:
# plot(mesh)
# plt.xlabel('X')
# plt.ylabel('Y')
# plt.show()
# if save:
#     plt.savefig(f'{line}mesh_mshr.png')
#     plt.savefig(f'{line}mesh_mshr.svg')

In [73]:
# Create functional space
F = FunctionSpace(mesh, polynom, eo)  # piecewise linear polynomials
V = VectorFunctionSpace(mesh, polynom, eo)

# Create 'trial function' (the unknown function to be approximated) over the space F
tem = TrialFunction(F)  # scalar temperature field, such that: tem(x,y) = Sum(tem_i* w(x,y))
d1 = tem.geometric_dimension()  # space dimension
u = TrialFunction(V)  # vector displacement field
d2 = u.geometric_dimension()  # space dimension


In [74]:
# Create scalar 'test function' (or 'shape function', or 'weighting function') over the space F
q = TestFunction(F)  # q -> tem
v = TestFunction(V)  # v -> u

t = 0
# Define expressions used in variational forms to prevent repeated code generation
n = FacetNormal(mesh)  # per-face normal vector
# k_ = (k / (rho * Cp))  # thermal conductivity
k_ = k  # thermal conductivity
Traction = Constant((0, 0))  # traction


In [75]:
# tem_surf = '(300 + ' \
#           '500 * exp(-pow( (t-5)/dt, 2)) * ' \
#          '( (( pow((x[0] - L_x/2)/r, 2) + pow((x[2] - L_z/2)/r, 2) ) <= 1.0) ? 1.0 : 0.0))'
tem_surf = '(THot * (x[1] == L_y))'
# tem_surf = '(40)'
tem_desired = Expression(tem_surf, L_y=L_y, THot = THot, degree=eo)
# fact_ = Constant(1.0e8)

In [76]:

# Problem settings
tem_0 = T0  # [K] initial temperature
tem_ref = TAir  # [K] reference temperature


In [77]:
# Define strains and stress
def epsilon(u):
    return 0.5 * (nabla_grad(u) + nabla_grad(u).T)
    # return sym(nabla_grad(u))


def epsilon_therm(tem):
    return alpha * (tem - tem_ref) * Identity(d1)


def sigma(u):
    return lambda_ * nabla_div(u) * Identity(d2) + 2 * mu * epsilon(u) \
        - (3 * lambda_ + 2 * mu) * epsilon_therm(tem)


# Compute boundary surfaces of the mesh (via C++ wrapper)

tol = 1E-14


# https://fenicsproject.org/olL_zocs/dolfin/1.3.0/python/demo/documented/subdomains-poisson/python/documentation.html
class Bottom(SubDomain):
    def inside(self, x, on_boundary):
        return near(x[2], 0.0)


class Top(SubDomain):
    def inside(self, x, on_boundary):
        return near(x[2], L_z)


In [78]:

# bottom surfcae
# bs_bottom_wall = 'near(x[1], 0.0)'
# bs_bottom_wall = near(x[1],0.0)
bs_bottom_wall = Bottom()
# bs_top_wall = 'near(x[1],L_y)'
# bs_top_wall = near(x[1],L_y)
bs_top_wall = Top()

# bs_top_edges = [Point(0.0, L_y), Point(L_x, L_y)]


# bs_top_edges = Point(0.0, L_y)

def left_top_fixed_point(x, on_boundary):
    tol = 1E-15
    return (near(x[0], 0.0) and near(x[1],0.0) and near(x[2],L_z))

# def left_top_fixed_point(x, on_boundary):
#     tol = 1E-15
#     return Point(0.0, L_y)


def right_top_fixed_point(x, on_boundary):
    tol = 1E-15
    return (near(x[0], L_x) and near(x[1],0.0) and near(x[2],L_z))


In [79]:
dx = Measure('dx', domain=mesh, subdomain_data=pd)


In [80]:

# BC at the bottom
# bc_tem_bottom = DirichletBC(F, TAir, bs_bottom_wall)
# bc_tem_top = DirichletBC(F, THot, bs_top_wall)

bc_tem_bottom = DirichletBC(F, THot, fd,27)
bc_tem_top = DirichletBC(F, TAir, fd,26)



#

# bcs_tem = [bc_tem_bottom]
bcs_tem = [bc_tem_bottom, bc_tem_top]
# bc_u_bottom = DirichletBC(V.sub(0), Constant((0,0)), bs_top_edges)
#
# bc_top_fixed_point1 = DirichletBC(V.sub(0), Constant(0), right_top_fixed_point, method='pointwise')
rt_y_fixed_point = DirichletBC(V.sub(1), Constant(0), right_top_fixed_point, method='pointwise')
rt_y_fixed_point = DirichletBC(V.sub(2), Constant(0), right_top_fixed_point, method='pointwise')

lt_x_fixed_point = DirichletBC(V.sub(0), Constant(0), left_top_fixed_point, method='pointwise')
lt_y_fixed_point = DirichletBC(V.sub(1), Constant(0), left_top_fixed_point, method='pointwise')
lt_y_fixed_point = DirichletBC(V.sub(2), Constant(0), left_top_fixed_point, method='pointwise')
#
# bcs_u = [bc_lower_fixed_point1,bc_lower_fixed_point2, bc_top_fixed_point1,bc_top_fixed_point2]
bcs_u = [lt_x_fixed_point, lt_y_fixed_point, rt_y_fixed_point]
# bcs_u = [bc_u_bottom]

In [81]:
# Compose weak form of PDE(heat):
tem_0 = Expression('T0', degree=eo, T0 = T0)
tem_n = project(tem_0, F, solver_type="cg", preconditioner_type="amg")
tem = TrialFunction(F)
q = TestFunction(F)

In [82]:
dS()

Measure('interior_facet', subdomain_id='everywhere')

In [83]:
dx(1)

Measure('cell', subdomain_id=1, domain=Mesh(VectorElement(FiniteElement('Lagrange', tetrahedron, 1), dim=3), 186), subdomain_data=<dolfin.cpp.mesh.MeshFunctionSizet object at 0x7f05c5f1ef70>)

In [84]:
dx(31)

Measure('cell', subdomain_id=31, domain=Mesh(VectorElement(FiniteElement('Lagrange', tetrahedron, 1), dim=3), 186), subdomain_data=<dolfin.cpp.mesh.MeshFunctionSizet object at 0x7f05c5f1ef70>)

In [85]:
dx

Measure('cell', subdomain_id='everywhere', domain=Mesh(VectorElement(FiniteElement('Lagrange', tetrahedron, 1), dim=3), 186), subdomain_data=<dolfin.cpp.mesh.MeshFunctionSizet object at 0x7f05c5f1ef70>)

In [86]:
dX

In [87]:
# fact_ = Constant(0.01)
# eq_tem = (k_ * inner(nabla_grad(tem), nabla_grad(q)) * dX + (tem_desired - tem) * fact_ * q * dX)
# eq_tem = (k_ * inner(nabla_grad(tem), nabla_grad(q)) * dX + alphaAir * (tem - TAir) * q * dX)
eq_tem = (k_ * inner(grad(tem), grad(q)) * dx )
# eq_tem = k1 * inner(grad(tem), grad(q)) * dx(31) + k2 * inner(grad(tem), grad(q)) * dx(32)
a_therm = lhs(eq_tem)
L_therm = rhs(eq_tem)


In [88]:
# Compose weak form of PDE(elasticity):
# Split the equation into right-hand-side (RHS) and left-hand-side (LHS) parts:
u = TrialFunction(V)
v = TestFunction(V)
sigma_ = lambda_ * nabla_div(u) * Identity(d2) + mu * (nabla_grad(u) + nabla_grad(u).T) \
         - (3 * lambda_ + 2 * mu) * alpha * (tem_n - tem_ref) * Identity(d1)
epsilon_ = 0.5 * (nabla_grad(v) + nabla_grad(v).T)
elastic_eqn = inner(sigma_, epsilon_) * dx
# LHS, bilinear form
a_elas = lhs(elastic_eqn)
# RHS, Linear form
L_elas = rhs(elastic_eqn)


In [89]:
if save:
    # Prepare export to file
    path = line
    tem_file = File(f'{path}tem.pvd')
    u_file = File(f'{path}u.pvd')
    vms_file = File(f'{path}vms.pvd')

In [90]:
# Time loop
tem = Function(F)
u = Function(V)

print(f'DOFs = {V.dim() + F.dim()}')


DOFs = 991624


In [91]:
# Update time value in expression
# tem_desired.t = t

sol_settings = {'linear_solver': 'mumps'}

# Compute HCE
solve(a_therm == L_therm, tem, bcs_tem, solver_parameters=sol_settings)

    Solving linear variational problem.


In [92]:

# Update previous solution
tem_n.assign(tem)

# Compute elasticity
solve(a_elas == L_elas, u, bcs_u, solver_parameters=sol_settings)


    Solving linear variational problem.


RuntimeError: 

*** -------------------------------------------------------------------------
*** DOLFIN encountered an error. If you are not able to resolve this issue
*** using the information listed below, you can ask for help at
***
***     fenics-support@googlegroups.com
***
*** Remember to include the error message listed below and, if possible,
*** include a *minimal* running example to reproduce the error.
***
*** -------------------------------------------------------------------------
*** Error:   Unable to solve linear system using PETSc Krylov solver.
*** Reason:  Solution failed to converge in 0 iterations (PETSc reason DIVERGED_PC_FAILED, residual norm ||r|| = 0.000000e+00).
*** Where:   This error was encountered inside PETScKrylovSolver.cpp.
*** Process: 0
*** 
*** DOLFIN version: 2019.1.0
*** Git changeset:  ba376b6aebd7a9bc089be46b50bdd9f5c548fb91
*** -------------------------------------------------------------------------


In [ ]:
# Compute stress
s = sigma(u) - (1. / 3) * tr(sigma(u)) * Identity(d2)  # deviatoric stress
von_Mises = sqrt(3. / 2 * inner(s, s))
von_Mises = project(von_Mises, F)


In [93]:
if save:
    # Save solution
    tem.rename('T [K]', 'label')
    tem_file << (tem)
    # u.rename('U [m]', 'label')
    # u_file << (u)
    # von_Mises.rename('VMS [Pa]', 'label')
    # vms_file << (von_Mises)

print(f'{t}')
# print(f'{t} / {dt*num_steps}')


0


In [94]:
from matplotlib import cm
fig = plot(tem,mesh)
fig.set_cmap(cm.rainbow)
plt.colorbar(fig)
plt.show()

TypeError: gca() got an unexpected keyword argument 'projection'

In [95]:
from matplotlib import cm
fig = plot(tem_n,mesh)
fig.set_cmap(cm.rainbow)

# gr = fig.axes.pcolor(tem.vector())
plt.colorbar(fig)
plt.show()
fig = plot(u,mesh)
plt.xlabel('X')
plt.ylabel('Y')
plt.xlim(-.002,0.022)
plt.ylim(-0.002,0.014)
plt.colorbar(fig)
if save:
    plt.savefig(f'{line}u_vec.png')
plt.show()

TypeError: gca() got an unexpected keyword argument 'projection'